In [ ]:
import sys
import os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import rcParams, cycler
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import xarray as xr
import geopandas as gpd
import rioxarray
from xeofs.xarray import EOF
import gc
import collections
import statsmodels.api as sm
import scipy
from sklearn.metrics import mean_squared_error
from math import sqrt
from itertools import product
import pyproj
from shapely.geometry import mapping
import pandas as pd

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.rc('font', family='sans-serif') 
plt.rc('font', serif='Helvetica Neue')

In [ ]:
# Define project repo path
inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'

# DATA FILE PATHS

# Data containing regridded flux and SSH for 150 years
regriddedFluxSSH_filepath = 'data/interim/RegriddedFluxSSH.nc'

# File contains all defined ice shelves
iceShelvesShape_filepath = 'data/interim/iceShelves.geojson'

# Folder for output figures
figures_folderpath = 'reports/figures/' 

interim_data_folder = 'data/interim/'
flux_dedrafted_data_path = 'dedrafted_flux_IMBIE/'
randomized_realizations_path = 'randomized_realizations/'
flux_dedrafted_iceshelves_data_path = 'iceshelves_dedrafted_flux/'
reconstructions_neofs_path = 'reconstructions_neofs/'
cise_file_path = 'cise_data/'

In [ ]:
# Read geoJSON region feature file as GeoDataFrame
iceshelvesmask = gpd.read_file(inDirName + iceShelvesShape_filepath)
# Convert to south polar stereographic projection
icems = iceshelvesmask.to_crs({'init': 'epsg:3031'});
crs = ccrs.SouthPolarStereo();

In [ ]:
# Input data files

regridded_data = xr.open_dataset(inDirName+regriddedFluxSSH_filepath)
flux = regridded_data.timeMonthly_avg_landIceFreshwaterFlux
ssh = regridded_data.timeMonthly_avg_ssh

flux_clean = xr.open_dataset(inDirName+interim_data_folder+'flux_clean')
flux_clean = flux_clean.timeMonthly_avg_landIceFreshwaterFlux

# change file name for other realizations
flux_reconstr0 = xr.open_dataset(inDirName+interim_data_folder+'flux_reconstr_0.nc')
flux_reconstr0 = flux_reconstr0.reconstructed_data

flux_reconstr1 = xr.open_dataset(inDirName+interim_data_folder+'flux_reconstr_1.nc')
flux_reconstr1 = flux_reconstr1.reconstructed_data

flux_reconstr2 = xr.open_dataset(inDirName+interim_data_folder+'flux_reconstr_2.nc')
flux_reconstr2 = flux_reconstr2.reconstructed_data

flux_reconstr3 = xr.open_dataset(inDirName+interim_data_folder+'flux_reconstr_3.nc')
flux_reconstr3 = flux_reconstr3.reconstructed_data

flux_reconstr4 = xr.open_dataset(inDirName+interim_data_folder+'flux_reconstr_4.nc')
flux_reconstr4 = flux_reconstr4.reconstructed_data

catchments_scatter = np.load(inDirName+interim_data_folder+"catchments_scatter.npy")
eofs = xr.open_dataset(inDirName+interim_data_folder+"fluxclean_eofs.nc")
pcs = xr.open_dataset(inDirName+interim_data_folder+"fluxclean_pcs.nc")
varexpl = xr.open_dataset(inDirName+interim_data_folder+"fluxclean_varexpl.nc")

In [ ]:
flux_clean_ts = flux_clean.sum(['y','x'])

In [ ]:
nmodes = model.n_modes

In [ ]:
# Figure: Anomalies / Internal Variability Component of Input Data: Temporal Variability
plt.figure(figsize=(45,5))
#plt.plot(flux_anm_ts)
plt.plot(flux_clean_ts[200:], color = '#44546A', linewidth=2)
plt.title('Anomalies / Internal Variability Component')
plt.ylabel('Freshwater Flux (kg/m2/s)')
plt.xlabel('Time (months)')

## Spatial Decomposition: EOFs

In [ ]:
%%time
model = EOF(flux_clean)
model.solve()
eofs = model.eofs()
pcs = model.pcs()
nmodes = model.n_modes
varexpl = model.explained_variance_ratio()

In [ ]:
#eofs.to_netcdf(inDirName+interim_data_folder+"fluxclean_eofs.nc")
#pcs.to_netcdf(inDirName+interim_data_folder+"fluxclean_pcs.nc")
#varexpl.to_netcdf(inDirName+interim_data_folder+"fluxclean_varexpl.nc")

In [ ]:
# Figure: Relative power of m EOFs

m = 40
plt.figure(figsize=(15,8))
plt.bar(range(m),varexpl[:40]*100, color='#44546A')
plt.ylabel('Relative Power (%)')
plt.xlabel('EOF Mode')
plt.title('Total Relative Power of first {} EOFs: {}'.format(m,varexpl[:m].sum().values*100))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(icems.name)

In [ ]:
snscolors = sns.color_palette("deep")
from matplotlib.projections import get_projection_class

In [ ]:
127-33

In [ ]:
# Figure: Melt rate draft dependence for select catchments
catchment_number = 1 # Select in [0,99]

catchments = np.array([1,27,28,61,70,83,94,71,22])

# Amery = 1
# George VI = 27
# Getz = 28
# Pine Island = 61
# Ronne = 70
# Totten = 83
# Wilkins = 94
# Western Ross = 71
# Filchner = 22
# Thwaites = 81
# Eastern_Ross = 72

#fig, ax = plt.subplots(1, 1, figsize=[15, 15],subplot_kw={'projection': ccrs.SouthPolarStereo()})
fig, axs = plt.subplots(3,3, figsize=[15, 15])
axs=axs.ravel()

#axins = inset_axes(ax, width="20%", height="20%")
#axins = axs[0].inset_axes([0.65, 0.6, 0.25, 0.25])

for i in range(9):
    axs[i].scatter(catchments_scatter[catchments[i]],ssh[1],color='r',s=4)
    axs[i].set_xlabel('Melt Flux (kg / m2 / s)')
    axs[i].set_ylabel('Depth (m)')
    axs[i].set_title(icems.name[33+catchments[i]])
    axins = axs[i].inset_axes([0.7, 0.7, 0.3, 0.3])
    icems[33:133].plot(ax=axins,linewidth=0.3)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='r')

# TODO: Polar Stereographic Projection for inset map
# TODO: Convert to m SLE units for flux

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize=[15, 15],subplot_kw={'projection': ccrs.SouthPolarStereo()})
fig, axs = plt.subplots(1,1, figsize=[15, 15])

#axins = inset_axes(ax, width="20%", height="20%")
#axins = axs[0].inset_axes([0.65, 0.6, 0.25, 0.25])

for i in range(9):
    axs.scatter(catchments_scatter[catchments[i]],ssh[1],color='r',s=4)
    axs.set_xlabel('Melt Flux (kg / m2 / s)')
    axs.set_ylabel('Depth (m)')
    axs.set_title(icems.name[33+catchments[i]])
    axins = axs.inset_axes([0.7, 0.7, 0.3, 0.3])
    icems[33:133].plot(ax=axins,linewidth=0.3)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='r')

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition

In [ ]:
#fig, axs = plt.subplots(1, 1, figsize=[15, 15],subplot_kw={'projection': ccrs.SouthPolarStereo()})
#ax = plt.subplot(111)
fig, ax = plt.subplots(1,1, figsize=[15, 8])

catchments = np.array([1,27,28,61,70,83,94,71,22])
#axins = inset_axes(ax, width="20%", height="20%")
#axins = axs[0].inset_axes([0.65, 0.6, 0.25, 0.25])

i = 0
ax.scatter(catchments_scatter[catchments[i]],ssh[1],color='r',s=4)
ax.set_xlabel('Melt Flux (kg / m2 / s)')
ax.set_ylabel('Depth (m)')
ax.set_title(icems.name[33+catchments[i]])
axins = ax.inset_axes([0.7, 0.7, 0.3, 0.3])
icems[33:133].plot(ax=axins,linewidth=0.3)
icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='r')

In [ ]:
plt.figure(figsize=(20,8))
ax1 = plt.subplot(121,projection=ccrs.SouthPolarStereo())
ax1.gridlines(color='whitesmoke',zorder=4)
icems[34:133].plot(ax=ax1,color='antiquewhite', linewidth=0,zorder=1)
icems[34:133].boundary.plot(ax=ax1,color='r', linewidth=0.2,zorder=3)
#icems[34:133].boundary.plot(ax=ax1,linewidth=0.25,color='lightgray',zorder=4)
#ax1.coastlines(resolution='10m', zorder=6,linewidth=0.75)
ax1.patch.set_facecolor(color='lightsteelblue')
#ax1.add_feature(cartopy.feature.LAND, color='ghostwhite')
plt.title('Catchment Boundaries');

In [ ]:
abs_max = np.amax(np.abs(eofs))
eof_norm = eofs*1.0/abs_max

In [ ]:
eofs.EOFs[:,:,1].plot()

In [ ]:
# Figure: Dominant EOF modes
fg = eof_norm[:,:,0:2].plot(aspect=2,
                            size=8,
                            col="mode", 
                            col_wrap=2,
                            subplot_kws=dict(projection=ccrs.SouthPolarStereo()))

# TODO: Redefine vmax, vmin 

## Fourier Phase Randomization

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 5
t_length = pcs.PCs.shape[0]

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.PCs.shape[0],pcs.PCs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs.PCs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_fourier = np.fft.rfft(fl)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
        print('calculated ifft for realization {}, mode: {}'.format(i,m))

In [ ]:
# FIGURE (FOR TESTING)

fig, axs = plt.subplots(2,2,figsize=(30,16))
axs=axs.ravel()

for mode_m in range(0,4):
    for i in range(0,5):
        axs[mode_m].psd(new_fl[i,:,mode_m],color='b', linewidth=0.5)
    axs[mode_m].psd(new_fl[0,:,mode_m],color='b', linewidth=0.25,label='Randomized')
    axs[mode_m].psd(pcs.PCs[:,mode_m],color='k', linewidth=1.5,label='Original')
    axs[mode_m].set_title('PSD of PC for mode {}'.format(mode_m))
plt.legend()

In [ ]:
#modified to not return f - in calculation of RMSE, only Px required
def psd_calc_grid(data,y,x):
    f, Px = scipy.signal.welch(data[:,y,x])
    return Px

In [ ]:
def time_series(clipped_data):
    clipped_ts = clipped_data.sum(['y','x'])
    return clipped_ts

In [ ]:
# Reconstruct flux dataset using phase randomized PCs
# This section is to be called iteratively for ensemble runs with multiple realizations
# This method also takes 'modes' as a parameter: 
# which is used to reconstruct dataset with different number of selected modes

def generate_data(n_realization,mode,mode_skip):
    flux_reconstr = model.reconstruct_randomized_X(new_fl[n_realization],slice(1,mode,mode_skip))
    #flux_reconstr = flux_reconstr.dropna('time',how='all')
    #flux_reconstr = flux_reconstr.dropna('y',how='all')
    #flux_reconstr = flux_reconstr.dropna('x',how='all')
    #flux_reconstr = flux_reconstr.drop("month")
    return flux_reconstr

In [ ]:
def clip_data(total_data, basin):
    clipped_data = total_data.rio.clip(icems.loc[[basin],'geometry'].apply(mapping))
    #clipped_data = clipped_data.dropna('time',how='all')
    #clipped_data = clipped_data.dropna('y',how='all')
    #clipped_data = clipped_data.dropna('x',how='all')
    clipped_data = clipped_data.drop("month")
    return clipped_data

In [ ]:
clipped_amery = clip_data(flux_clean, 34)

In [ ]:
# Generate dataset realizations
for i in range(n_realizations):
    flux_reconstr = generate_data(i, 65, 1)
    flux_reconstr.to_netcdf(inDirName+interim_data_folder+cise_file_path+"flux_reconstr_65_{}.nc".format(i))
    del flux_reconstr
    gc.collect()
    print('reconstructed realization # {}'.format(i))

In [ ]:
basins = np.array((34,55,114,103,104,105))

In [ ]:
for basin in basins:
    orig = clip_data(flux_clean, basin)
    rec0 = clip_data(flux_reconstr0, basin)
    rec1 = clip_data(flux_reconstr1, basin)
    rec2 = clip_data(flux_reconstr2, basin)
    rec3 = clip_data(flux_reconstr3, basin)
    rec4 = clip_data(flux_reconstr4, basin)
    orig.to_netcdf(inDirName+interim_data_folder+cise_file_path+"orig_m65_{}".format(basin))
    rec0.to_netcdf(inDirName+interim_data_folder+cise_file_path+"rec0_m65_{}".format(basin))
    rec1.to_netcdf(inDirName+interim_data_folder+cise_file_path+"rec1_m65_{}".format(basin))
    rec2.to_netcdf(inDirName+interim_data_folder+cise_file_path+"rec2_m65_{}".format(basin))
    rec3.to_netcdf(inDirName+interim_data_folder+cise_file_path+"rec3_m65_{}".format(basin))
    rec4.to_netcdf(inDirName+interim_data_folder+cise_file_path+"rec4_m65_{}".format(basin))
    del orig, rec0, rec1, rec2, rec3, rec4
    gc.collect()

In [ ]:
# Amery 34
orig_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_34")
rec0_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_34")
rec1_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_34")
rec2_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_34")
rec3_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_34")
rec4_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_34")

# Filchner 55
orig_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_55")
rec0_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_55")
rec1_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_55")
rec2_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_55")
rec3_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_55")
rec4_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_55")

# Thwaites 114
orig_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_114")
rec0_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_114")
rec1_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_114")
rec2_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_114")
rec3_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_114")
rec4_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_114")

# Ronne 103
orig_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_103")
rec0_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_103")
rec1_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_103")
rec2_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_103")
rec3_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_103")
rec4_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_103")

# Western Ross 104
orig_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_104")
rec0_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_104")
rec1_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_104")
rec2_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_104")
rec3_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_104")
rec4_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_104")

# Eastern Ross 105
orig_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_105")
rec0_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_105")
rec1_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_105")
rec2_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_105")
rec3_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_105")
rec4_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_105")

In [ ]:
# Amery 34
orig_m65_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_m65_34")
rec0_m65_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_m65_34")
rec1_m65_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_m65_34")
rec2_m65_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_m65_34")
rec3_m65_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_m65_34")
rec4_m65_34 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_m65_34")

# Filchner 55
orig_m65_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_m65_55")
rec0_m65_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_m65_55")
rec1_m65_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_m65_55")
rec2_m65_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_m65_55")
rec3_m65_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_m65_55")
rec4_m65_55 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_m65_55")

# Thwaites 114
orig_m65_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_m65_114")
rec0_m65_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_m65_114")
rec1_m65_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_m65_114")
rec2_m65_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_m65_114")
rec3_m65_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_m65_114")
rec4_m65_114 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_m65_114")

# Ronne 103
orig_m65_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_m65_103")
rec0_m65_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_m65_103")
rec1_m65_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_m65_103")
rec2_m65_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_m65_103")
rec3_m65_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_m65_103")
rec4_m65_103 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_m65_103")

# Western Ross 104
orig_m65_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_m65_104")
rec0_m65_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_m65_104")
rec1_m65_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_m65_104")
rec2_m65_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_m65_104")
rec3_m65_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_m65_104")
rec4_m65_104 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_m65_104")

# Eastern Ross 105
orig_m65_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"orig_m65_105")
rec0_m65_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec0_m65_105")
rec1_m65_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec1_m65_105")
rec2_m65_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec2_m65_105")
rec3_m65_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec3_m65_105")
rec4_m65_105 = xr.open_dataset(inDirName+interim_data_folder+cise_file_path+"rec4_m65_105")

In [ ]:
# Amery 
orig_m65_34_ts = time_series(orig_m65_34)
rec0_m65_34_ts = time_series(rec0_m65_34)
rec1_m65_34_ts = time_series(rec1_m65_34)
rec2_m65_34_ts = time_series(rec2_m65_34)
rec3_m65_34_ts = time_series(rec3_m65_34)
rec4_m65_34_ts = time_series(rec4_m65_34)

# Filchner 
orig_m65_55_ts = time_series(orig_m65_55)
rec0_m65_55_ts = time_series(rec0_m65_55)
rec1_m65_55_ts = time_series(rec1_m65_55)
rec2_m65_55_ts = time_series(rec2_m65_55)
rec3_m65_55_ts = time_series(rec3_m65_55)
rec4_m65_55_ts = time_series(rec4_m65_55)

# Thwaites 
orig_m65_114_ts = time_series(orig_m65_114)
rec0_m65_114_ts = time_series(rec0_m65_114)
rec1_m65_114_ts = time_series(rec1_m65_114)
rec2_m65_114_ts = time_series(rec2_m65_114)
rec3_m65_114_ts = time_series(rec3_m65_114)
rec4_m65_114_ts = time_series(rec4_m65_114)

# Ronne 
orig_m65_103_ts = time_series(orig_m65_103)
rec0_m65_103_ts = time_series(rec0_m65_103)
rec1_m65_103_ts = time_series(rec1_m65_103)
rec2_m65_103_ts = time_series(rec2_m65_103)
rec3_m65_103_ts = time_series(rec3_m65_103)
rec4_m65_103_ts = time_series(rec4_m65_103)

# Western Ross 
orig_m65_104_ts = time_series(orig_m65_104)
rec0_m65_104_ts = time_series(rec0_m65_104)
rec1_m65_104_ts = time_series(rec1_m65_104)
rec2_m65_104_ts = time_series(rec2_m65_104)
rec3_m65_104_ts = time_series(rec3_m65_104)
rec4_m65_104_ts = time_series(rec4_m65_104)

# Eastern Ross 
orig_m65_105_ts = time_series(orig_m65_105)
rec0_m65_105_ts = time_series(rec0_m65_105)
rec1_m65_105_ts = time_series(rec1_m65_105)
rec2_m65_105_ts = time_series(rec2_m65_105)
rec3_m65_105_ts = time_series(rec3_m65_105)
rec4_m65_105_ts = time_series(rec4_m65_105)

In [ ]:
fig, axs = plt.subplots(6,1,figsize=(30,24))
axs=axs.ravel()

axs[0].plot(orig_34_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[0].plot(rec0_34_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[0].plot(rec1_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].plot(rec2_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].plot(rec3_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].plot(rec4_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].set_title('Catchment: Amery');

axs[1].plot(orig_55_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[1].plot(rec0_55_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[1].plot(rec1_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].plot(rec2_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].plot(rec3_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].plot(rec4_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].set_title('Catchment: Filchner');

axs[2].plot(orig_114_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[2].plot(rec0_114_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[2].plot(rec1_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].plot(rec2_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].plot(rec3_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].plot(rec4_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].set_title('Catchment: Thwaites');

axs[3].plot(orig_103_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[3].plot(rec0_103_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[3].plot(rec1_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].plot(rec2_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].plot(rec3_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].plot(rec4_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].set_title('Catchment: Ronne');

axs[4].plot(orig_104_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[4].plot(rec0_104_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[4].plot(rec1_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].plot(rec2_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].plot(rec3_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].plot(rec4_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].set_title('Catchment: Western Ross');

axs[5].plot(orig_105_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[5].plot(rec0_105_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[5].plot(rec1_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].plot(rec2_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].plot(rec3_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].plot(rec4_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].set_title('Catchment: Eastern Ross');
plt.legend()

plt.suptitle('Realizations Generated using 1500 EOF modes');

In [ ]:
fig, axs = plt.subplots(6,1,figsize=(30,24))
axs=axs.ravel()

axs[0].plot(orig_m65_34_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[0].plot(rec0_m65_34_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[0].plot(rec1_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].plot(rec2_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].plot(rec3_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].plot(rec4_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].set_title('Catchment: Amery');

axs[1].plot(orig_m65_55_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[1].plot(rec0_m65_55_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[1].plot(rec1_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].plot(rec2_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].plot(rec3_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].plot(rec4_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].set_title('Catchment: Filchner');

axs[2].plot(orig_m65_114_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[2].plot(rec0_m65_114_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[2].plot(rec1_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].plot(rec2_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].plot(rec3_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].plot(rec4_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].set_title('Catchment: Thwaites');

axs[3].plot(orig_m65_103_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[3].plot(rec0_m65_103_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[3].plot(rec1_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].plot(rec2_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].plot(rec3_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].plot(rec4_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].set_title('Catchment: Ronne');

axs[4].plot(orig_m65_104_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[4].plot(rec0_m65_104_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[4].plot(rec1_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].plot(rec2_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].plot(rec3_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].plot(rec4_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].set_title('Catchment: Western Ross');

axs[5].plot(orig_m65_105_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original',zorder=10);
axs[5].plot(rec0_m65_105_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[5].plot(rec1_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].plot(rec2_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].plot(rec3_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].plot(rec4_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].set_title('Catchment: Eastern Ross');
plt.legend()

plt.suptitle('Realizations Generated using 65 EOF modes');

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(30,18))
axs=axs.ravel()

axs[0].psd(orig_m65_34_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original');
axs[0].psd(rec0_m65_34_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[0].psd(rec1_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].psd(rec2_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].psd(rec3_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].psd(rec4_m65_34_ts.reconstructed_data,color='b', linewidth=0.5);
axs[0].set_title('Catchment: Amery');

axs[1].psd(orig_m65_55_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original');
axs[1].psd(rec0_m65_55_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[1].psd(rec1_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].psd(rec2_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].psd(rec3_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].psd(rec4_m65_55_ts.reconstructed_data,color='b', linewidth=0.5);
axs[1].set_title('Catchment: Filchner');

axs[2].psd(orig_m65_114_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original');
axs[2].psd(rec0_m65_114_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[2].psd(rec1_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].psd(rec2_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].psd(rec3_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].psd(rec4_m65_114_ts.reconstructed_data,color='b', linewidth=0.5);
axs[2].set_title('Catchment: Thwaites');

axs[3].psd(orig_m65_103_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original');
axs[3].psd(rec0_m65_103_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[3].psd(rec1_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].psd(rec2_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].psd(rec3_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].psd(rec4_m65_103_ts.reconstructed_data,color='b', linewidth=0.5);
axs[3].set_title('Catchment: Ronne');

axs[4].psd(orig_m65_104_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original');
axs[4].psd(rec0_m65_104_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[4].psd(rec1_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].psd(rec2_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].psd(rec3_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].psd(rec4_m65_104_ts.reconstructed_data,color='b', linewidth=0.5);
axs[4].set_title('Catchment: Western Ross');

axs[5].psd(orig_m65_105_ts.timeMonthly_avg_landIceFreshwaterFlux,color='k', linewidth=1.5,label='Original');
axs[5].psd(rec0_m65_105_ts.reconstructed_data,color='b', linewidth=0.5,label='Generated');
axs[5].psd(rec1_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].psd(rec2_m65_105_ts.reconstructed_data,color='b', linewidth=0.5)t;
axs[5].psd(rec3_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].psd(rec4_m65_105_ts.reconstructed_data,color='b', linewidth=0.5);
axs[5].set_title('Catchment: Eastern Ross');
plt.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[6, 6],subplot_kw={'projection': ccrs.SouthPolarStereo()})
ax.coastlines(resolution='10m', zorder=6,linewidth=0.75)
orig_34.timeMonthly_avg_landIceFreshwaterFlux[500].plot(ax=ax)